# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [0]:
# 打開瀏覽器
# !pip install selenium
# !apt install chromium-chromedriver

from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('chromedriver',options=options)
browser.maximize_window()

from bs4 import BeautifulSoup
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")

In [2]:
# 解析網頁-監測站
select =  soup.find(id="ctl05_ddlSite")
# print(select)


sites = []
for option in select.find_all('option'):
  # print(option['value'],option.text)
  sites.append([option['value'],option.text])

import pandas as pd
df_site = pd.DataFrame(sites,columns=['site_number','site_name'])
df_site

,site_number,site_name
0,11,臺北市-士林
1,16,臺北市-大同
2,12,臺北市-中山
3,14,臺北市-古亭
4,15,臺北市-松山
...,...,...
72,80,臺東縣-關山
73,63,花蓮縣-花蓮
74,78,澎湖縣-馬公
75,75,連江縣-馬祖


### ① 取出 台北市士林區 2018 年度監測資料

In [0]:
# 模擬使用者操作行為，選擇/點擊

site = 29      # 台北市士林區
year = 2018

from selenium.webdriver.support.ui import Select
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))

selectSite.select_by_value(f'{site}')
selectYear.select_by_value(f'{year}')

browser.find_element_by_id('ctl05_btnQuery').click()


In [4]:
# 解析網頁
html_source1 = browser.page_source
soup1 = BeautifulSoup(html_source1, "html5lib")
table = soup1.find('table', class_='TABLE_G')
# print(table)

d = {}

datas = table.find_all('tr')[1:]

for i in datas:
    if(len(i.find_all('td'))) == 5:
            
        param = i.find_all('td')[0].get_text()              # 監測項目
        d[param] = {}
           
        month = i.find_all('td')[2].get_text()              # 監測日期

        d[param][month] = i.find_all('td')[3].get_text()    # 監測值
        # print(param,month)

    else:
        date = i.find_all('td')[0].get_text()                # 監測日期

        d[param][date] = i.find_all('td')[1].get_text()      # 監測值
        # print(date)
      

import pandas as pd
df_datas = pd.DataFrame(d)
df_datas[:-1]

,SO2,CO,O3,PM10,NOx,NO,NO2,THC,NMHC,CH4
2018/01,1.90,0.37,28.90,39,17.15,3.17,13.98,NaN,NaN,NaN
2018/02,1.90,0.40,32.50,55,17.57,2.81,14.76,NaN,NaN,NaN
2018/03,2.50,0.38,36.70,58,18.45,2.22,16.22,NaN,NaN,NaN
2018/04,3.50,0.40,40.70,60,17.07,1.92,15.15,NaN,NaN,NaN
2018/05,3,0.28,29.80,30,13.40,1.81,11.59,NaN,NaN,NaN
2018/06,3,0.27,32.60,28,12.93,1.99,10.93,NaN,NaN,NaN
2018/07,3,0.23,25.80,29,11.50,1.88,9.63,NaN,NaN,NaN
2018/08,2.20,0.27,27.60,26,12.16,2.15,10,NaN,NaN,NaN
2018/09,2.80,0.26,34.10,38,13.07,1.82,11.24,NaN,NaN,NaN
2018/10,2.20,0.30,45.80,39,12.72,1.40,11.33,NaN,NaN,NaN


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [5]:
df_datas.loc['2018/01':'2018/08',['SO2','CO']]

,SO2,CO
2018/01,1.90,0.37
2018/02,1.90,0.40
2018/03,2.50,0.38
2018/04,3.50,0.40
2018/05,3,0.28
2018/06,3,0.27
2018/07,3,0.23
2018/08,2.20,0.27
